# EDA (PM10, PM2,5)

*alpha 1.0*

* PM10, PM2,5, O3, NO2, SO2, C6H6, CO - data from GIOŚ archive: https://powietrze.gios.gov.pl/pjp/archives
* Prediction will be based on weather (last 24h and current - forcast for predicting future pollutant values) - data from IMGW archive: https://dane.imgw.pl/data/dane_pomiarowo_obserwacyjne/dane_meteorologiczne/terminowe/synop/
* weather forcast / current params - https://openweathermap.org/api
* All pollutants are time series, 1 measure per hour
* time period used: last 4 years (2016, 2017, 2018, 2019)

## env

In [76]:
import os
import sys

import pandas as pd
pd.options.display.max_rows = 25
pd.options.display.max_columns = None
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns

# 2019

In [2]:
pd.read_excel('../data/gios/2019_PM10_1g.xlsx').head(10)

,Nr,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139
0,Kod stacji,DsDusznikMOB,DsDzialoszyn,DsDziePilsud,DsJaworMOB,DsJelGorOgin,DsKlodzSzkol,DsLegAlRzecz,DsLubanMieszMOB,DsNowRudJezi,DsOlawZolnAK,DsWalbrzWyso,DsWrocWybCon,DsZabkPowWar,KpBydPlPozna,KpBydWarszaw,KpInowSolank,KpMogiNowMOB,KpToruDziewu,KpToruKaszow,KpToruWSikor,KpWloclKalis,KpWloclOkrze,LbBiaPodOrze,LbChelPolan,LbLubObywate,LbNaleczow,LbZamoHrubie,LdGajewUjWod,LdLodzCzerni,LdLodzGdansk,LdLodzJanPaw,LdPabiKonsta,LdPioTrKraPr,LdRadomsRoln,LdZgieMielcz,LuGorzKosGdy,LuNowaSolMOB,LuSulecDudka,LuWsKaziWiel,LuZarySzyman,LuZielKrotka,MpKrakAlKras,MpKrakBujaka,MpKrakBulwar,MpKrakDietla,MpKrakOsPias,MpKrakSwoszo,MpKrakWadow,MpKrakZloRog,MpNoSaczNadb,MpNoTargPSlo,MpSkawOsOgro,MpSuchaNiesz,MpTarBitStud,MpTarRoSitko,MpTrzebOsZWM,MpZakopaSien,MzKonJezMos,MzLegZegrzyn,MzMinMazKaziMOB,MzOtwoBrzozo,MzPiasPulask,MzPlocMiReja,MzRadTochter,MzSiedKonars,MzWarAlNiepo,MzWarBajkowa,MzWarChrosci,MzWarKondrat,MzWarTolstoj,MzWarWokalna,MzZyraRoosev,OpOlesSlowac,OpOpoleKoszy,OpPrudPodgor,OpZdziePiast,PdBialWarsza,PdBorsukowiz,PdLomSikorsk,PkDebiGrottg,PkHorZdrParkMOB,PkJarosPruch,PkJasloSikor,PkMielBierna,PkNiskoSzkla,PkPrzemGrunw,PkRymZdrPark,PkRzeszPilsu,PkRzeszRejta,PmGdaKacze02,PmGdaLeczk08,PmGdaPoWie01,PmGdaWyzwo03,PmGdyPoreb04,PmGdySzaf09N,PmKosTargo12,PmLebMalcz16,PmMalMicki15,PmSlupKniazi,PmSopBitPl06,SkKielTargow,SkMaloSlonec,SkNowiParkow,SkPolaRuszcz,SkSkarZielnaMOB,SkStaraZlota,SlBielKossak,SlCiesMickie,SlCzestoArmK,SlCzestoBacz,SlDabro1000L,SlGliwicMewy,SlKatoKossut,SlLublSzymal,SlRybniBorki,SlSosnoLubel,SlTychyTolst,SlUstronSana,SlWodzGalczy,SlZabSkloCur,SlZawGalczyn,SlZlotPotLes,SlZywieKoper,WmElbBazynsk,WmElkStadion,WmGoldJacwie,WmGoldUzdrowMOB,WmOlsPuszkin,WmOstrPilsud,WpBoroDrapal,WpKoniWyszyn,WpKozieosLes,WpPoznDabrow,WpPoznRatajeMOB,ZpKoszArKraj,ZpSzczAndr01,ZpSzczBudzWosMOB,ZpSzczecPrze,ZpSzczPils02
1,Wskaźnik,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10
2,Czas uśredniania,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g
3,Jednostka,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,u

In [7]:
df_2019 = pd.read_excel('../data/gios/2019_PM10_1g.xlsx', header=5, index_col=0)

In [8]:
df_2019.head(10)

,DsDusznikMOB-PM10-1g,DsDzialoszyn-PM10-1g,DsDziePilsud-PM10-1g,DsJaworMOB-PM10-1g,DsJelGorOgin-PM10-1g,DsKlodzSzkol-PM10-1g,DsLegAlRzecz-PM10-1g,DsLubanMieszMOB-PM10-1g,DsNowRudJezi-PM10-1g,DsOlawZolnAK-PM10-1g,DsWalbrzWyso-PM10-1g,DsWrocWybCon-PM10-1g,DsZabkPowWar-PM10-1g,KpBydPlPozna-PM10-1g,KpBydWarszaw-PM10-1g,KpInowSolank-PM10-1g,KpMogiNowMOB-PM10-1g,KpToruDziewu-PM10-1g,KpToruKaszow-PM10-1g,KpToruWSikor-PM10-1g,KpWloclKalis-PM10-1g,KpWloclOkrze-PM10-1g,LbBiaPodOrze-PM10-1g,LbChelPolan-PM10-1g,LbLubObywate-PM10-1g,LbNaleczow-PM10-1g,LbZamoHrubie-PM10-1g,LdGajewUjWod-PM10-1g,LdLodzCzerni-PM10-1g,LdLodzGdansk-PM10-1g,LdLodzJanPaw-PM10-1g,LdPabiKonsta-PM10-1g,LdPioTrKraPr-PM10-1g,LdRadomsRoln-PM10-1g,LdZgieMielcz-PM10-1g,LuGorzKosGdy-PM10-1g,LuNowaSolMOB-PM10-1g,LuSulecDudka-PM10-1g,LuWsKaziWiel-PM10-1g,LuZarySzyman-PM10-1g,LuZielKrotka-PM10-1g,MpKrakAlKras-PM10-1g,MpKrakBujaka-PM10-1g,MpKrakBulwar-PM10-1g,MpKrakDietla-PM10-1g,MpKrakOsPias-PM10-1g,MpKrakSwoszo-PM10-1g,MpKrakWadow-PM10-1g,MpKrakZloRog-PM10-1g,MpNoSaczNadb-PM10-1g,MpNoTargPSlo-PM10-1g,MpSkawOsOgro-PM10-1g,MpSuchaNiesz-PM10-1g,MpTarBitStud-PM10-1g,MpTarRoSitko-PM10-1g,MpTrzebOsZWM-PM10-1g,MpZakopaSien-PM10-1g,MzKonJezMos-PM10-1g,MzLegZegrzyn-PM10-1g,MzMinMazKaziMOB-PM10-1g,MzOtwoBrzozo-PM10-1g,MzPiasPulask-PM10-1g,MzPlocMiReja-PM10-1g,MzRadTochter-PM10-1g,MzSiedKonars-PM10-1g,MzWarAlNiepo-PM10-1g,MzWarBajkowa-PM10-1g,MzWarChrosci-PM10-1g,MzWarKondrat-PM10-1g,MzWarTolstoj-PM10-1g,MzWarWokalna-PM10-1g,MzZyraRoosev-PM10-1g,OpOlesSlowac-PM10-1g,OpOpoleKoszy-PM10-1g,OpPrudPodgor-PM10-1g,OpZdziePiast-PM10-1g,PdBialWarsza-PM10-1g,PdBorsukowiz-PM10-1g,PdLomSikorsk-PM10-1g,PkDebiGrottg-PM10-1g,PkHorZdrParkMOB-PM10-1g,PkJarosPruch-PM10-1g,PkJasloSikor-PM10-1g,PkMielBierna-PM10-1g,PkNiskoSzkla-PM10-1g,PkPrzemGrunw-PM10-1g,PkRymZdrPark-PM10-1g,PkRzeszPilsu-PM10-1g,PkRzeszRejta-PM10-1g,PmGdaKacze02-PM10-1g,PmGdaLeczk08-PM10-1g,PmGdaPoWie01-PM10-1g,PmGdaWyzwo03-PM10-1g,PmGdyPoreb04-PM10-1g,PmGdySzaf09N-PM10-1g,PmKosTargo12-PM10-1g,PmLebMalcz16-PM10-1g,PmMalMicki15-PM10-1g,PmSlupKniazi-PM10-1g,PmSopBitPl06-PM10-1g,SkKielTargow-PM10-1g,SkMaloSlonec-PM10-1g,SkNowiParkow-PM10-1g,SkPolaRuszcz-PM10-1g,SkSkarZielnaMOB-PM10-1g,SkStaraZlota-PM10-1g,SlBielKossak-PM10-1g,SlCiesMickie-PM10-1g,SlCzestoArmK-PM10-1g,SlCzestoBacz-PM10-1g,SlDabro1000L-PM10-1g,SlGliwicMewy-PM10-1g,SlKatoKossut-PM10-1g,SlLublSzymal-PM10-1g,SlRybniBorki-PM10-1g,SlSosnoLubel-PM10-1g,SlTychyTolst-PM10-1g,SlUstronSana-PM10-1g,SlWodzGalczy-PM10-1g,SlZabSkloCur-PM10-1g,SlZawGalczyn-PM10-1g,SlZlotPotLes-PM10-1g,SlZywieKoper-PM10-1g,WmElbBazynsk-PM10-1g,WmElkStadion-PM10-1g,WmGoldJacwie-PM10-1g,WmGoldUzdrowMOB-PM10-1g,WmOlsPuszkin-PM10-1g,WmOstrPilsud-PM10-1g,WpBoroDrapal-PM10-1g,WpKoniWyszyn-PM10-1g,WpKozieosLes-PM10-1g,WpPoznDabrow-PM10-1g,WpPoznRatajeMOB-PM10-1g,ZpKoszArKraj-PM10-1g,ZpSzczAndr01-PM10-1g,ZpSzczBudzWosMOB-PM10-1g,ZpSzczecPrze-PM10-1g,ZpSzczPils02-PM10-1g
Kod stanowiska,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2019-01-01 01:00:00,35.21080,16.68570,245.92900,55.28140,125.32600,306.4300,65.3853,31.1785,NaN,39.710,35.29540,115.5170,46.40820,94.5,114.2649,15.1050,NaN,86.496,NaN,26.7,NaN,53.753,NaN,14.0,65.3200,NaN,99.7014,38.0,70.0,162.294,35.0,53.0,42.0,37.0,69.040,57.03110,76.34730,17.4035,58.2072,152.5210,36.9684,88.2185,139.7900,120.0570,73.4495,161.0530,NaN,83.6841,77.8355,63.7522,132.9710,NaN,72.2746,67.43160,73.1073,113.0970,138.17600,39.357284,48.155374,NaN,39.772007,NaN,38.098950,40.389135,69.48,73.077490,59.870348,62.6603,54.113472,NaN,46.527342,48.724218,24.2266,38.08460,NaN,50.4629,43.8,8.5,76.0,45.1,NaN,36.6,23.4,69.4362,36.4,50.4,19.3,35.9147,31.1,23.78530,25.73020,76.30860,16.39110,34.67610,22.19170,56.37780,69.17820,25.67840,NaN,24.03820,50.0,31.72,25.80,18.78,NaN,32.69112,169.5440,62.6338,83.2158,58.4966,119.2360,130.7090,84.9443,60.1095,63.3477,55.9938,54.9406,20.58670,175.2180,151.9340,NaN

In [10]:
df_2019.index.name ='Data pomiaru'

In [11]:
df_2019.head(10)

,DsDusznikMOB-PM10-1g,DsDzialoszyn-PM10-1g,DsDziePilsud-PM10-1g,DsJaworMOB-PM10-1g,DsJelGorOgin-PM10-1g,DsKlodzSzkol-PM10-1g,DsLegAlRzecz-PM10-1g,DsLubanMieszMOB-PM10-1g,DsNowRudJezi-PM10-1g,DsOlawZolnAK-PM10-1g,DsWalbrzWyso-PM10-1g,DsWrocWybCon-PM10-1g,DsZabkPowWar-PM10-1g,KpBydPlPozna-PM10-1g,KpBydWarszaw-PM10-1g,KpInowSolank-PM10-1g,KpMogiNowMOB-PM10-1g,KpToruDziewu-PM10-1g,KpToruKaszow-PM10-1g,KpToruWSikor-PM10-1g,KpWloclKalis-PM10-1g,KpWloclOkrze-PM10-1g,LbBiaPodOrze-PM10-1g,LbChelPolan-PM10-1g,LbLubObywate-PM10-1g,LbNaleczow-PM10-1g,LbZamoHrubie-PM10-1g,LdGajewUjWod-PM10-1g,LdLodzCzerni-PM10-1g,LdLodzGdansk-PM10-1g,LdLodzJanPaw-PM10-1g,LdPabiKonsta-PM10-1g,LdPioTrKraPr-PM10-1g,LdRadomsRoln-PM10-1g,LdZgieMielcz-PM10-1g,LuGorzKosGdy-PM10-1g,LuNowaSolMOB-PM10-1g,LuSulecDudka-PM10-1g,LuWsKaziWiel-PM10-1g,LuZarySzyman-PM10-1g,LuZielKrotka-PM10-1g,MpKrakAlKras-PM10-1g,MpKrakBujaka-PM10-1g,MpKrakBulwar-PM10-1g,MpKrakDietla-PM10-1g,MpKrakOsPias-PM10-1g,MpKrakSwoszo-PM10-1g,MpKrakWadow-PM10-1g,MpKrakZloRog-PM10-1g,MpNoSaczNadb-PM10-1g,MpNoTargPSlo-PM10-1g,MpSkawOsOgro-PM10-1g,MpSuchaNiesz-PM10-1g,MpTarBitStud-PM10-1g,MpTarRoSitko-PM10-1g,MpTrzebOsZWM-PM10-1g,MpZakopaSien-PM10-1g,MzKonJezMos-PM10-1g,MzLegZegrzyn-PM10-1g,MzMinMazKaziMOB-PM10-1g,MzOtwoBrzozo-PM10-1g,MzPiasPulask-PM10-1g,MzPlocMiReja-PM10-1g,MzRadTochter-PM10-1g,MzSiedKonars-PM10-1g,MzWarAlNiepo-PM10-1g,MzWarBajkowa-PM10-1g,MzWarChrosci-PM10-1g,MzWarKondrat-PM10-1g,MzWarTolstoj-PM10-1g,MzWarWokalna-PM10-1g,MzZyraRoosev-PM10-1g,OpOlesSlowac-PM10-1g,OpOpoleKoszy-PM10-1g,OpPrudPodgor-PM10-1g,OpZdziePiast-PM10-1g,PdBialWarsza-PM10-1g,PdBorsukowiz-PM10-1g,PdLomSikorsk-PM10-1g,PkDebiGrottg-PM10-1g,PkHorZdrParkMOB-PM10-1g,PkJarosPruch-PM10-1g,PkJasloSikor-PM10-1g,PkMielBierna-PM10-1g,PkNiskoSzkla-PM10-1g,PkPrzemGrunw-PM10-1g,PkRymZdrPark-PM10-1g,PkRzeszPilsu-PM10-1g,PkRzeszRejta-PM10-1g,PmGdaKacze02-PM10-1g,PmGdaLeczk08-PM10-1g,PmGdaPoWie01-PM10-1g,PmGdaWyzwo03-PM10-1g,PmGdyPoreb04-PM10-1g,PmGdySzaf09N-PM10-1g,PmKosTargo12-PM10-1g,PmLebMalcz16-PM10-1g,PmMalMicki15-PM10-1g,PmSlupKniazi-PM10-1g,PmSopBitPl06-PM10-1g,SkKielTargow-PM10-1g,SkMaloSlonec-PM10-1g,SkNowiParkow-PM10-1g,SkPolaRuszcz-PM10-1g,SkSkarZielnaMOB-PM10-1g,SkStaraZlota-PM10-1g,SlBielKossak-PM10-1g,SlCiesMickie-PM10-1g,SlCzestoArmK-PM10-1g,SlCzestoBacz-PM10-1g,SlDabro1000L-PM10-1g,SlGliwicMewy-PM10-1g,SlKatoKossut-PM10-1g,SlLublSzymal-PM10-1g,SlRybniBorki-PM10-1g,SlSosnoLubel-PM10-1g,SlTychyTolst-PM10-1g,SlUstronSana-PM10-1g,SlWodzGalczy-PM10-1g,SlZabSkloCur-PM10-1g,SlZawGalczyn-PM10-1g,SlZlotPotLes-PM10-1g,SlZywieKoper-PM10-1g,WmElbBazynsk-PM10-1g,WmElkStadion-PM10-1g,WmGoldJacwie-PM10-1g,WmGoldUzdrowMOB-PM10-1g,WmOlsPuszkin-PM10-1g,WmOstrPilsud-PM10-1g,WpBoroDrapal-PM10-1g,WpKoniWyszyn-PM10-1g,WpKozieosLes-PM10-1g,WpPoznDabrow-PM10-1g,WpPoznRatajeMOB-PM10-1g,ZpKoszArKraj-PM10-1g,ZpSzczAndr01-PM10-1g,ZpSzczBudzWosMOB-PM10-1g,ZpSzczecPrze-PM10-1g,ZpSzczPils02-PM10-1g
Data pomiaru,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2019-01-01 01:00:00,35.21080,16.68570,245.92900,55.28140,125.32600,306.4300,65.3853,31.1785,NaN,39.710,35.29540,115.5170,46.40820,94.5,114.2649,15.1050,NaN,86.496,NaN,26.7,NaN,53.753,NaN,14.0,65.3200,NaN,99.7014,38.0,70.0,162.294,35.0,53.0,42.0,37.0,69.040,57.03110,76.34730,17.4035,58.2072,152.5210,36.9684,88.2185,139.7900,120.0570,73.4495,161.0530,NaN,83.6841,77.8355,63.7522,132.9710,NaN,72.2746,67.43160,73.1073,113.0970,138.17600,39.357284,48.155374,NaN,39.772007,NaN,38.098950,40.389135,69.48,73.077490,59.870348,62.6603,54.113472,NaN,46.527342,48.724218,24.2266,38.08460,NaN,50.4629,43.8,8.5,76.0,45.1,NaN,36.6,23.4,69.4362,36.4,50.4,19.3,35.9147,31.1,23.78530,25.73020,76.30860,16.39110,34.67610,22.19170,56.37780,69.17820,25.67840,NaN,24.03820,50.0,31.72,25.80,18.78,NaN,32.69112,169.5440,62.6338,83.2158,58.4966,119.2360,130.7090,84.9443,60.1095,63.3477,55.9938,54.9406,20.58670,175.2180,151.9340,NaN,2

In [5]:
df_2019.shape

(8760, 140)

In [6]:
24*365

8760

In [17]:
df_2019.head(10)

,DsDusznikMOB-PM10-1g,DsDzialoszyn-PM10-1g,DsDziePilsud-PM10-1g,DsJaworMOB-PM10-1g,DsJelGorOgin-PM10-1g,DsKlodzSzkol-PM10-1g,DsLegAlRzecz-PM10-1g,DsLubanMieszMOB-PM10-1g,DsNowRudJezi-PM10-1g,DsOlawZolnAK-PM10-1g,DsWalbrzWyso-PM10-1g,DsWrocWybCon-PM10-1g,DsZabkPowWar-PM10-1g,KpBydPlPozna-PM10-1g,KpBydWarszaw-PM10-1g,KpInowSolank-PM10-1g,KpMogiNowMOB-PM10-1g,KpToruDziewu-PM10-1g,KpToruKaszow-PM10-1g,KpToruWSikor-PM10-1g,KpWloclKalis-PM10-1g,KpWloclOkrze-PM10-1g,LbBiaPodOrze-PM10-1g,LbChelPolan-PM10-1g,LbLubObywate-PM10-1g,LbNaleczow-PM10-1g,LbZamoHrubie-PM10-1g,LdGajewUjWod-PM10-1g,LdLodzCzerni-PM10-1g,LdLodzGdansk-PM10-1g,LdLodzJanPaw-PM10-1g,LdPabiKonsta-PM10-1g,LdPioTrKraPr-PM10-1g,LdRadomsRoln-PM10-1g,LdZgieMielcz-PM10-1g,LuGorzKosGdy-PM10-1g,LuNowaSolMOB-PM10-1g,LuSulecDudka-PM10-1g,LuWsKaziWiel-PM10-1g,LuZarySzyman-PM10-1g,LuZielKrotka-PM10-1g,MpKrakAlKras-PM10-1g,MpKrakBujaka-PM10-1g,MpKrakBulwar-PM10-1g,MpKrakDietla-PM10-1g,MpKrakOsPias-PM10-1g,MpKrakSwoszo-PM10-1g,MpKrakWadow-PM10-1g,MpKrakZloRog-PM10-1g,MpNoSaczNadb-PM10-1g,MpNoTargPSlo-PM10-1g,MpSkawOsOgro-PM10-1g,MpSuchaNiesz-PM10-1g,MpTarBitStud-PM10-1g,MpTarRoSitko-PM10-1g,MpTrzebOsZWM-PM10-1g,MpZakopaSien-PM10-1g,MzKonJezMos-PM10-1g,MzLegZegrzyn-PM10-1g,MzMinMazKaziMOB-PM10-1g,MzOtwoBrzozo-PM10-1g,MzPiasPulask-PM10-1g,MzPlocMiReja-PM10-1g,MzRadTochter-PM10-1g,MzSiedKonars-PM10-1g,MzWarAlNiepo-PM10-1g,MzWarBajkowa-PM10-1g,MzWarChrosci-PM10-1g,MzWarKondrat-PM10-1g,MzWarTolstoj-PM10-1g,MzWarWokalna-PM10-1g,MzZyraRoosev-PM10-1g,OpOlesSlowac-PM10-1g,OpOpoleKoszy-PM10-1g,OpPrudPodgor-PM10-1g,OpZdziePiast-PM10-1g,PdBialWarsza-PM10-1g,PdBorsukowiz-PM10-1g,PdLomSikorsk-PM10-1g,PkDebiGrottg-PM10-1g,PkHorZdrParkMOB-PM10-1g,PkJarosPruch-PM10-1g,PkJasloSikor-PM10-1g,PkMielBierna-PM10-1g,PkNiskoSzkla-PM10-1g,PkPrzemGrunw-PM10-1g,PkRymZdrPark-PM10-1g,PkRzeszPilsu-PM10-1g,PkRzeszRejta-PM10-1g,PmGdaKacze02-PM10-1g,PmGdaLeczk08-PM10-1g,PmGdaPoWie01-PM10-1g,PmGdaWyzwo03-PM10-1g,PmGdyPoreb04-PM10-1g,PmGdySzaf09N-PM10-1g,PmKosTargo12-PM10-1g,PmLebMalcz16-PM10-1g,PmMalMicki15-PM10-1g,PmSlupKniazi-PM10-1g,PmSopBitPl06-PM10-1g,SkKielTargow-PM10-1g,SkMaloSlonec-PM10-1g,SkNowiParkow-PM10-1g,SkPolaRuszcz-PM10-1g,SkSkarZielnaMOB-PM10-1g,SkStaraZlota-PM10-1g,SlBielKossak-PM10-1g,SlCiesMickie-PM10-1g,SlCzestoArmK-PM10-1g,SlCzestoBacz-PM10-1g,SlDabro1000L-PM10-1g,SlGliwicMewy-PM10-1g,SlKatoKossut-PM10-1g,SlLublSzymal-PM10-1g,SlRybniBorki-PM10-1g,SlSosnoLubel-PM10-1g,SlTychyTolst-PM10-1g,SlUstronSana-PM10-1g,SlWodzGalczy-PM10-1g,SlZabSkloCur-PM10-1g,SlZawGalczyn-PM10-1g,SlZlotPotLes-PM10-1g,SlZywieKoper-PM10-1g,WmElbBazynsk-PM10-1g,WmElkStadion-PM10-1g,WmGoldJacwie-PM10-1g,WmGoldUzdrowMOB-PM10-1g,WmOlsPuszkin-PM10-1g,WmOstrPilsud-PM10-1g,WpBoroDrapal-PM10-1g,WpKoniWyszyn-PM10-1g,WpKozieosLes-PM10-1g,WpPoznDabrow-PM10-1g,WpPoznRatajeMOB-PM10-1g,ZpKoszArKraj-PM10-1g,ZpSzczAndr01-PM10-1g,ZpSzczBudzWosMOB-PM10-1g,ZpSzczecPrze-PM10-1g,ZpSzczPils02-PM10-1g
Data pomiaru,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2019-01-01 01:00:00,35.21080,16.68570,245.92900,55.28140,125.32600,306.4300,65.3853,31.1785,NaN,39.710,35.29540,115.5170,46.40820,94.5,114.2649,15.1050,NaN,86.496,NaN,26.7,NaN,53.753,NaN,14.0,65.3200,NaN,99.7014,38.0,70.0,162.294,35.0,53.0,42.0,37.0,69.040,57.03110,76.34730,17.4035,58.2072,152.5210,36.9684,88.2185,139.7900,120.0570,73.4495,161.0530,NaN,83.6841,77.8355,63.7522,132.9710,NaN,72.2746,67.43160,73.1073,113.0970,138.17600,39.357284,48.155374,NaN,39.772007,NaN,38.098950,40.389135,69.48,73.077490,59.870348,62.6603,54.113472,NaN,46.527342,48.724218,24.2266,38.08460,NaN,50.4629,43.8,8.5,76.0,45.1,NaN,36.6,23.4,69.4362,36.4,50.4,19.3,35.9147,31.1,23.78530,25.73020,76.30860,16.39110,34.67610,22.19170,56.37780,69.17820,25.67840,NaN,24.03820,50.0,31.72,25.80,18.78,NaN,32.69112,169.5440,62.6338,83.2158,58.4966,119.2360,130.7090,84.9443,60.1095,63.3477,55.9938,54.9406,20.58670,175.2180,151.9340,NaN,2

In [93]:
index_mz_2019 = df_2019.columns.str.count('MzWar*') > 0 #only stations in Warsaw
index_mz_2019

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False,  True,  True,  True,  True,  True,  True, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False,

In [41]:
pm10_2019 = df_2019.iloc[:, index_mz_2019]
pm10_2019

,MzWarAlNiepo-PM10-1g,MzWarBajkowa-PM10-1g,MzWarChrosci-PM10-1g,MzWarKondrat-PM10-1g,MzWarTolstoj-PM10-1g,MzWarWokalna-PM10-1g
Data pomiaru,,,,,,
2019-01-01 01:00:00,73.077490,59.870348,62.6603,54.113472,NaN,46.527342
2019-01-01 02:00:00,50.048078,41.162822,43.1520,33.890919,NaN,32.420843
2019-01-01 03:00:00,36.705641,33.912537,25.8013,26.994749,NaN,24.564058
2019-01-01 04:00:00,28.115305,28.362936,20.3174,19.539430,NaN,19.475005
2019-01-01 05:00:00,28.663624,27.557349,21.1265,18.887089,NaN,19.475005
...,...,...,...,...,...,...
2019-12-31 20:00:00,15.229800,8.937552,10.8779,7.704111,7.11,7.154138
2019-12-31 21:00:00,19.068036,9.171183,12.2264,5.653898,7.32,7.421983
2019-12-31 22:00:00,14.590095,8.119844,9.9789,5.094749,4.59,5.725632


In [44]:
pm10_2019.shape #both number and names match the list of stations on GIOS website

(8760, 6)

In [64]:
pm10_2019.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 8760 entries, 2019-01-01 01:00:00 to 2020-01-01 00:00:00
Data columns (total 6 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   MzWarAlNiepo-PM10-1g  8555 non-null   float64
 1   MzWarBajkowa-PM10-1g  8733 non-null   float64
 2   MzWarChrosci-PM10-1g  8711 non-null   float64
 3   MzWarKondrat-PM10-1g  8709 non-null   float64
 4   MzWarTolstoj-PM10-1g  8306 non-null   float64
 5   MzWarWokalna-PM10-1g  8212 non-null   float64
dtypes: float64(6)
memory usage: 479.1 KB


In [79]:
pm10_2019.isna().sum()

MzWarAlNiepo-PM10-1g    205
MzWarBajkowa-PM10-1g     27
MzWarChrosci-PM10-1g     49
MzWarKondrat-PM10-1g     51
MzWarTolstoj-PM10-1g    454
MzWarWokalna-PM10-1g    548
dtype: int64

# 2018

In [50]:
pd.read_excel('../data/gios/2018_PM10_1g.xlsx').head(10)

,Unnamed: 0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126
0,NaN,DsDzialoszyn,DsDziePilsud,DsJelGorOgin,DsKlodzSzkol,DsLadekMOB,DsLegAlRzecz,DsLubanMOB,DsNowRudSreb,DsOlawZolnAK,DsWalbrzWyso,DsWrocWybCon,DsZabkPowWar,KpBydPlPozna,KpBydWarszaw,KpGrudPilsud,KpInowSolank,KpToruDziewu,KpToruKaszow,KpToruStorczMOB,KpToruWSikor,KpWloclOkrze,LbLubObywate,LbZamoHrubie,LdGajewUjWod,LdLodzCzerni,LdLodzGdansk,LdLodzJanPaw,LdPabiKonsta,LdPioTrKraPr,LdRadomsRoln,LdZgieMielcz,LuGorzKosGdy,LuSulecDudka,LuWsKaziWiel,LuZarySzyman,LuZielKrotka,MpKrakAlKras,MpKrakBujaka,MpKrakBulwar,MpKrakDietla,MpKrakOsPias,MpKrakTelime,MpKrakWadow,MpKrakZloRog,MpNoSaczNadb,MpNoTargPSlo,MpOlkuFrNull,MpSkawOsOgro,MpSuchaNiesz,MpTarBitStud,MpTarRoSitko,MpTrzebOsZWM,MpZakopaSien,MzKonJezMos,MzOtwoBrzozo,MzPlocMiReja,MzRadTochter,MzSiedKonars,MzWarAlNiepo,MzWarKondrat,MzWarTolstoj,MzWarWokalna,MzZyraRoosev,OpKKozBSmial,OpOlesSlowac,OpOpoleKoszy,OpZdziePiast,PdAugustoZdrMOB,PdBialWarsza,PdLomSikorsk,PkJasloSikor,PkMielBierna,PkNiskoSzkla,PkPrzemGrunw,PkRymZdrPark,PkRzeszRejta,PmGdaKacze02,PmGdaLeczk08,PmGdaPoWie01,PmGdaWyzwo03,PmGdyPoreb04,PmGdySzaf09N,PmGdyWendy10,PmKosTargo12,PmLebMalcz16,PmMalMicki15,PmSlupKniazi,PmSopBitPl06,SkKielJagiel,SkKielTargow,SkKonsGranatMOB,SkMaloSlonec,SkNowiParkow,SkPolaRuszcz,SlBielKossak,SlCiesMickie,SlCzestoArmK,SlCzestoBacz,SlDabro1000L,SlGliwicMewy,SlKatoKossut,SlLublSzymal,SlRybniBorki,SlSosnoLubel,SlTychyTolst,SlUstronSana,SlWodzGalczy,SlZabSkloCur,SlZlotPotLes,SlZywieKoper,WmElbBazynsk,WmElkStadion,WmGoldJacwie,WmOlsPuszkin,WmOstrPilsud,WpBoroDrapal,WpKaliSawick,WpKoniWyszyn,WpKozieosLes,WpPilaKusoci,WpPoznDabrow,WpPoznPolank,ZpKoszArKraj,ZpSzczAndr01,ZpSzczecPrze,ZpSzczLacz04
1,NaN,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10
2,NaN,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g
3,NaN,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3
4,Czas pomiaru,DsDzialoszyn-PM10-1g,DsDziePilsud-PM10-1g,DsJelGorOgin-PM10-1g,DsKlodzSzkol-PM10-1g,DsLadekMOB-PM10-1g,DsLegAlRzecz-PM10-1g,DsLubanMOB-PM10-1g,DsNow

In [53]:
pm10_2018 = pd.read_excel('../data/gios/2018_PM10_1g.xlsx', header=5, index_col=0)
pm10_2018

,DsDzialoszyn-PM10-1g,DsDziePilsud-PM10-1g,DsJelGorOgin-PM10-1g,DsKlodzSzkol-PM10-1g,DsLadekMOB-PM10-1g,DsLegAlRzecz-PM10-1g,DsLubanMOB-PM10-1g,DsNowRudSreb-PM10-1g,DsOlawZolnAK-PM10-1g,DsWalbrzWyso-PM10-1g,DsWrocWybCon-PM10-1g,DsZabkPowWar-PM10-1g,KpBydPlPozna-PM10-1g,KpBydWarszaw-PM10-1g,KpGrudPilsud-PM10-1g,KpInowSolank-PM10-1g,KpToruDziewu-PM10-1g,KpToruKaszow-PM10-1g,KpToruStorczMOB-PM10-1g,KpToruWSikor-PM10-1g,KpWloclOkrze-PM10-1g,LbLubObywate-PM10-1g,LbZamoHrubie-PM10-1g,LdGajewUjWod-PM10-1g,LdLodzCzerni-PM10-1g,LdLodzGdansk-PM10-1g,LdLodzJanPaw-PM10-1g,LdPabiKonsta-PM10-1g,LdPioTrKraPr-PM10-1g,LdRadomsRoln-PM10-1g,LdZgieMielcz-PM10-1g,LuGorzKosGdy-PM10-1g,LuSulecDudka-PM10-1g,LuWsKaziWiel-PM10-1g,LuZarySzyman-PM10-1g,LuZielKrotka-PM10-1g,MpKrakAlKras-PM10-1g,MpKrakBujaka-PM10-1g,MpKrakBulwar-PM10-1g,MpKrakDietla-PM10-1g,MpKrakOsPias-PM10-1g,MpKrakTelime-PM10-1g,MpKrakWadow-PM10-1g,MpKrakZloRog-PM10-1g,MpNoSaczNadb-PM10-1g,MpNoTargPSlo-PM10-1g,MpOlkuFrNull-PM10-1g,MpSkawOsOgro-PM10-1g,MpSuchaNiesz-PM10-1g,MpTarBitStud-PM10-1g,MpTarRoSitko-PM10-1g,MpTrzebOsZWM-PM10-1g,MpZakopaSien-PM10-1g,MzKonJezMos-PM10-1g,MzOtwoBrzozo-PM10-1g,MzPlocMiReja-PM10-1g,MzRadTochter-PM10-1g,MzSiedKonars-PM10-1g,MzWarAlNiepo-PM10-1g,MzWarKondrat-PM10-1g,MzWarTolstoj-PM10-1g,MzWarWokalna-PM10-1g,MzZyraRoosev-PM10-1g,OpKKozBSmial-PM10-1g,OpOlesSlowac-PM10-1g,OpOpoleKoszy-PM10-1g,OpZdziePiast-PM10-1g,PdAugustoZdrMOB-PM10-1g,PdBialWarsza-PM10-1g,PdLomSikorsk-PM10-1g,PkJasloSikor-PM10-1g,PkMielBierna-PM10-1g,PkNiskoSzkla-PM10-1g,PkPrzemGrunw-PM10-1g,PkRymZdrPark-PM10-1g,PkRzeszRejta-PM10-1g,PmGdaKacze02-PM10-1g,PmGdaLeczk08-PM10-1g,PmGdaPoWie01-PM10-1g,PmGdaWyzwo03-PM10-1g,PmGdyPoreb04-PM10-1g,PmGdySzaf09N-PM10-1g,PmGdyWendy10-PM10-1g,PmKosTargo12-PM10-1g,PmLebMalcz16-PM10-1g,PmMalMicki15-PM10-1g,PmSlupKniazi-PM10-1g,PmSopBitPl06-PM10-1g,SkKielJagiel-PM10-1g,SkKielTargow-PM10-1g,SkKonsGranatMOB-PM10-1g,SkMaloSlonec-PM10-1g,SkNowiParkow-PM10-1g,SkPolaRuszcz-PM10-1g,SlBielKossak-PM10-1g,SlCiesMickie-PM10-1g,SlCzestoArmK-PM10-1g,SlCzestoBacz-PM10-1g,SlDabro1000L-PM10-1g,SlGliwicMewy-PM10-1g,SlKatoKossut-PM10-1g,SlLublSzymal-PM10-1g,SlRybniBorki-PM10-1g,SlSosnoLubel-PM10-1g,SlTychyTolst-PM10-1g,SlUstronSana-PM10-1g,SlWodzGalczy-PM10-1g,SlZabSkloCur-PM10-1g,SlZlotPotLes-PM10-1g,SlZywieKoper-PM10-1g,WmElbBazynsk-PM10-1g,WmElkStadion-PM10-1g,WmGoldJacwie-PM10-1g,WmOlsPuszkin-PM10-1g,WmOstrPilsud-PM10-1g,WpBoroDrapal-PM10-1g,WpKaliSawick-PM10-1g,WpKoniWyszyn-PM10-1g,WpKozieosLes-PM10-1g,WpPilaKusoci-PM10-1g,WpPoznDabrow-PM10-1g,WpPoznPolank-PM10-1g,ZpKoszArKraj-PM10-1g,ZpSzczAndr01-PM10-1g,ZpSzczecPrze-PM10-1g,ZpSzczLacz04-PM10-1g
Czas pomiaru,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2018-01-01 01:00:00,"20,3","7,6677","43,8925","133,86",NaN,"23,5187",NaN,"63,9445",NaN,"8,49835","70,2096","25,545",NaN,"73,3474","43,9","14,1768","47,6156","62,1",NaN,"63,1","43,4951","40,9","41,3","26,0","80,0","58,195","19,083","41,561","58,264","19,046","100,37","51,8373","32,6172","20,2247","44,7033","33,5716","59,0589",NaN,"163,369","115,384","225,213","65,7993","36,3942","293,804","181,148","95,4599","28,6891","50,8524","137,471","31,8724","23,4586","67,5688","403,547","41,250269","146,42","36,810694","87,884161","92,028885","60,302232","117,73181","89,71","88,66","40,27838","25,4633","18,5786","21,6271","50,4017","15,7","33,1","48,5","16,53","24,7633","16,6543","15,6617","15,9474","13,8647","41,7469","95,9831","131,33",NaN,"64,8332","30,4472","71,2833","170,326","78,258","43,94","18,8475","20,6635","26,0",NaN,NaN,"29,23","20,57","25,5","23,5761","38,4976","71,0937","60,2816","63,4578","64,5325","122,531",NaN,"18,8547","54,5011","21,376","25,168","77,134","83,3165","30,1093","47,6886","82,6642","28,3106","16,9653","45,9985","47,2772","12,4215","35,872",NaN,"32,4526","26,8099","21,9038","30,128","37,4186","28,3564","44,5816","19,4714"
2018-01-01 02:00:00,"29,29","6,2681","8,63003","38,7

In [92]:
index_mz_2018 = pm10_2018.columns.str.count('MzWar*') > 0 #only stations in Warsaw
index_mz_2018

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False,  True,  True,  True,  True, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False])

In [59]:
pm10_2018.iloc[:, index_mz_2018]

,MzKonJezMos-PM10-1g,MzOtwoBrzozo-PM10-1g,MzPlocMiReja-PM10-1g,MzRadTochter-PM10-1g,MzSiedKonars-PM10-1g,MzWarAlNiepo-PM10-1g,MzWarKondrat-PM10-1g,MzWarTolstoj-PM10-1g,MzWarWokalna-PM10-1g,MzZyraRoosev-PM10-1g
Czas pomiaru,,,,,,,,,,
2018-01-01 01:00:00,"41,250269","146,42","36,810694","87,884161","92,028885","60,302232","117,73181","89,71","88,66","40,27838"
2018-01-01 02:00:00,"32,731341","87,53","20,891538","38,992187","53,427057","39,993304","36,057921","26,64","41,4","29,29879"
2018-01-01 03:00:00,"26,833621","60,15","17,486863","36,653391","47,803956","34,437946","24,05336","23,33","31,91","27,51578"
2018-01-01 04:00:00,"23,276267","47,27","15,370444","38,101217","42,484807","32,070089","22,55279","22,19","29,68","26,389668"
2018-01-01 05:00:00,"19,999756","33,71","15,002371","33,200883","37,621585","30,430804","21,909689","19,14","26,37","24,700501"
...,...,...,...,...,...,...,...,...,...,...
2018-12-31 20:00:00,"37,131227","71,78","21,443648","27,855064","19,79","29,975446","30,698742",NaN,"21,85","30,987958"
2018-12-31 21:00:00,"34,884477","87,84","24,48025","30,305231","14,85","28,245089","19,337283",NaN,"19,53","31,738699"
2018-12-31 22:00:00,"24,680486","65,68","30,185381","32,198542","25,19","27,061161","17,086428",NaN,"25,75","24,418973"


In [65]:
pm10_2019.columns

Index(['MzWarAlNiepo-PM10-1g', 'MzWarBajkowa-PM10-1g', 'MzWarChrosci-PM10-1g',
       'MzWarKondrat-PM10-1g', 'MzWarTolstoj-PM10-1g', 'MzWarWokalna-PM10-1g'],
      dtype='object')

In [69]:
mz_cols = pm10_2018.iloc[:, index_mz_2018].columns
mz_cols

Index(['MzKonJezMos-PM10-1g', 'MzOtwoBrzozo-PM10-1g', 'MzPlocMiReja-PM10-1g',
       'MzRadTochter-PM10-1g', 'MzSiedKonars-PM10-1g', 'MzWarAlNiepo-PM10-1g',
       'MzWarKondrat-PM10-1g', 'MzWarTolstoj-PM10-1g', 'MzWarWokalna-PM10-1g',
       'MzZyraRoosev-PM10-1g'],
      dtype='object')

In [77]:
set(pm10_2019.columns) - set(mz_cols) #those two stations were built in 2018, date of first measurment: 2019-01-01

{'MzWarBajkowa-PM10-1g', 'MzWarChrosci-PM10-1g'}